# BoostCredit ETL Pipeline Demo

This notebook demonstrates the ETL pipeline for processing CSV and JSON data.

## Pipeline Flow:
1. **Extract** → Read data from CSV/JSON files
2. **Transform** → Clean, convert types, mask PII
3. **Store** → Save to object store (Parquet)
4. **Load** → Load from object store to PostgreSQL database

In [ ]:
import os
import sys
import importlib
import pandas as pd
from pathlib import Path

# Reload modules to ensure we have the latest code (important for notebooks)
import src.loaders
import src.pipeline
import src.extractors
import src.transformers
importlib.reload(src.loaders)
importlib.reload(src.pipeline)
importlib.reload(src.extractors)
importlib.reload(src.transformers)

from src.pipeline import Pipeline
from src.extractors import CSVExtractor, JSONExtractor
from src.transformers import CSVTransformer, JSONTransformer

# Set environment variables
os.environ['STORE_KEY'] = 'demo_data'
os.environ['DB_TYPE'] = 'postgresql'
os.environ['DB_HOST'] = 'localhost'
os.environ['DB_PORT'] = '5432'
os.environ['DB_USER'] = 'etl_user'
os.environ['DB_PASSWORD'] = 'etl_password'
os.environ['DB_NAME'] = 'etl_database'
os.environ['DATA_PATH'] = './data'
os.environ['OBJECT_STORE_PATH'] = './output'

print("✓ Environment variables set")
print("✓ Modules reloaded and imports successful")

✓ Environment variables set
✓ Imports successful


## Step 1: Initialize Pipeline

The pipeline handles the complete ETL process automatically.

In [7]:
pipeline = Pipeline()
print("✓ Pipeline initialized")

TypeError: SQLLoader._build_connection_string() missing 6 required positional arguments: 'db_type', 'db_user', 'db_password', 'db_host', 'db_port', and 'db_name'

## Step 2: Test Individual Components

Let's test each component separately to understand what they do.

In [ ]:
# Test CSV Extractor
csv_extractor = CSVExtractor()
csv_file = Path('data/test.csv')
if csv_file.exists():
    sample_data = csv_extractor.extract(str(csv_file))
    print(f"✓ CSV Extracted: {len(sample_data)} rows")
    print(f"  Columns: {list(sample_data.columns)}")
    print(f"\n  First row sample:")
    print(sample_data.head(1))
else:
    print("⚠ CSV file not found")

In [ ]:
# Test CSV Transformer
csv_transformer = CSVTransformer()
if csv_file.exists():
    transformed = csv_transformer.transform(sample_data.head(5))
    print("✓ CSV Transformed")
    print(f"  Data types converted")
    print(f"  PII masked (name, address)")
    print(f"\n  Transformed sample:")
    print(transformed[['id', 'name', 'created_at', 'is_claimed', 'paid_amount']].head(2))

In [ ]:
# Test JSON Extractor
json_extractor = JSONExtractor()
json_file = Path('data/test.json')
if json_file.exists():
    json_data = json_extractor.extract(str(json_file))
    print(f"✓ JSON Extracted: {len(json_data)} records")
    print(f"\n  First record keys: {list(json_data[0].keys())}")
    print(f"  Sample user_id: {json_data[0].get('user_id', 'N/A')}")
else:
    print("⚠ JSON file not found")

In [ ]:
# Test JSON Transformer
json_transformer = JSONTransformer()
if json_file.exists():
    json_transformed = json_transformer.transform(json_data[:2])  # Transform 2 records
    print("✓ JSON Transformed into 3 tables:")
    print(f"  - users: {len(json_transformed['users'])} rows")
    print(f"  - telephone_numbers: {len(json_transformed['telephone_numbers'])} rows")
    print(f"  - jobs_history: {len(json_transformed['jobs_history'])} rows")
    print(f"\n  Users sample:")
    print(json_transformed['users'][['user_id', 'name', 'username']].head(2))

## Step 3: Run Complete Pipeline

Now let's run the full pipeline for CSV processing.

In [ ]:
# Process CSV file
if csv_file.exists():
    os.environ['STORE_KEY'] = 'csv_demo'
    pipeline.process_csv('test.csv')
    print("✓ CSV processing completed!")
    print("  → Data extracted, transformed, saved to object store, and loaded to database")
else:
    print("⚠ CSV file not found - skipping CSV processing")

## Step 4: Process JSON File

Process JSON data which creates multiple linked tables.

In [ ]:
# Process JSON file
if json_file.exists():
    os.environ['STORE_KEY'] = 'json_demo'
    pipeline.process_json('test.json')
    print("✓ JSON processing completed!")
    print("  → Created 3 tables: users, telephone_numbers, jobs_history")
    print("  → All PII masked (emails, phones, national IDs, passwords)")
else:
    print("⚠ JSON file not found - skipping JSON processing")

## Step 5: Verify Data in Object Store

Check what was saved to the object store (intermediate step).

In [ ]:
from src.storage import ObjectStore

store = ObjectStore('./output')

# Check CSV data in store
csv_data = store.load('csv_demo', 'parquet')
if csv_data is not None:
    print(f"✓ CSV data in object store: {len(csv_data)} rows")
    print(f"  Columns: {list(csv_data.columns)}")

# Check JSON data in store
json_data_store = store.load('json_demo', 'parquet')
if json_data_store is not None:
    print(f"\n✓ JSON data in object store:")
    for table_name, df in json_data_store.items():
        print(f"  - {table_name}: {len(df)} rows")

## Step 6: Cleanup

Close the pipeline to release database connections.

In [ ]:
pipeline.close()
print("✓ Pipeline closed - database connections released")

ImportError: cannot import name 'ETLPipeline' from 'src.pipeline' (/home/anser/Downloads/anserGithub/BoostCredit/src/pipeline.py)

CSV file exists: True
CSV file size: 465.68 MB

First 5 rows:
     id              name                                            address  \
0  6311    Jennifer Green  7593 Juan Throughway Apt. 948\nWest Corey, TX ...   
1  3350      Karen Grimes    60975 Jessica Squares\nEast Sallybury, FL 71671   
2  9031       Calvin Cook                   PSC 3989, Box 4719\nAPO AA 42056   
3  1131    Peter Mcdowell                   PSC 1868, Box 4833\nAPO AP 77807   
4  1889  Mr. Ryan Sanchez      352 Simmons Circle\nPort Dustinbury, OK 83627   

    color               created_at  last_login is_claimed  paid_amount  
0    lime  Monday, June 30th, 2013  1202190735       True  5004.671532  
1    lime  Monday, June 30th, 2013   195884769       True   893.404595  
2  silver           1986-06-23TEST   623477862       True   266.600000  
3    aqua           1998-07-17TEST  1244885561       True   674.544127  
4   white      2006-05-09 13:29:58  1293151276      truee          NaN  

Columns: ['id', 'n

,id,name,address,color,created_at,last_login,is_claimed,paid_amount
0,6311,Jennifer Green,"7593 Juan Throughway Apt. 948\nWest Corey, TX ...",lime,"Monday, June 30th, 2013",1202190735,True,5004.671532
1,3350,Karen Grimes,"60975 Jessica Squares\nEast Sallybury, FL 71671",lime,"Monday, June 30th, 2013",195884769,True,893.404595
2,9031,Calvin Cook,"PSC 3989, Box 4719\nAPO AA 42056",silver,1986-06-23TEST,623477862,True,266.600000
3,1131,Peter Mcdowell,"PSC 1868, Box 4833\nAPO AP 77807",aqua,1998-07-17TEST,1244885561,True,674.544127
4,1889,Mr. Ryan Sanchez,"352 Simmons Circle\nPort Dustinbury, OK 83627",white,2006-05-09 13:29:58,1293151276,truee,NaN


NameError: name 'Path' is not defined